In [3]:
import pandas as pd
from google.cloud import bigquery
from geojson import Feature, FeatureCollection, dumps
from shapely import wkt
from shapely.geometry import mapping

# Instantiate the BigQuery client
client = bigquery.Client()

# Modify the SQL query to select only fire perimeter data, fire year, and fire name
query = """
SELECT fire_name, fire_year, ST_AsText(fires.geog) as geometry
FROM zestyai.atlas.wildland_fire_perimeters_all fires
"""

# Run the query and store the results in a dataframe
query_job = client.query(query)  # Make an API request.
df = query_job.to_dataframe()

# Convert the dataframe to a list of GeoJSON features
features = []
for index, row in df.iterrows():
    geometry = wkt.loads(row["geometry"])
    feature = Feature(
        geometry=mapping(geometry),
        properties={
            "fire_name": row["fire_name"],
            "fire_year": row["fire_year"]
        }
    )
    features.append(feature)

# Create a GeoJSON FeatureCollection
feature_collection = FeatureCollection(features)

# Convert the FeatureCollection to a GeoJSON string
geojson = dumps(feature_collection)

# Save the GeoJSON data to a file
with open("fire_perimeter_data.geojson", "w") as f:
    f.write(geojson)

print("GeoJSON data saved to 'fire_perimeter_data.geojson'")

GeoJSON data saved to 'fire_perimeter_data.geojson'
